In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 46 Lecture 2 Assignment

In this assignment, we will calculate a similarity matrix using responses to a survey about student life at a university. We will perform clustering on this dataset later on.

In [2]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist, squareform
from scipy.special import comb

import matplotlib.pyplot as plt

%matplotlib inline

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
data_url = "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv"
df = pd.read_csv(data_url)
df

,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,...,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,...,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,...,1,1,0,1,0,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,1,0,0,0,1,1,1,1,0,0,...,1,1,0,0,1,0,0,0,0,2954
2954,1,0,0,0,0,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,2955
2955,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2956
2956,0,1,0,1,1,1,1,1,0,1,...,1,0,0,0,0,1,1,0,0,2957


In [12]:
data_url = "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv"
df = pd.read_csv(data_url)
q5 = df.filter(like="Q5")
q5.head()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0


We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

The pdist function is a simple way to accomplish this goal, and the squareform function will transform the output of the pdist function into a tabular format that can be convert to a dataframe more easily.

In [13]:
dist = squareform(pdist(q5, "hamming"))
dist_df = pd.DataFrame(dist)
dist_df

,0,1,2,3,4,5,6,7,8,9,...,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,...,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,...,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,...,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,0.2,0.2,0.3,0.6,0.3,0.3,0.2,0.5,0.4,0.4,...,0.1,0.1,0.4,0.3,0.5,0.0,0.2,0.2,0.4,0.2
2954,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2955,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2956,0.2,0.2,0.1,0.6,0.5,0.3,0.2,0.3,0.6,0.4,...,0.3,0.5,0.4,0.5,0.5,0.4,0.2,0.2,0.0,0.2


Print the answers to the following three questions:

1. How many pairs of students are there in our dataset? (Remember to ignore a student pairing with themselves, and avoid double-counting - '3-5' is the same as '5-3'.) 
2. Of these pairs, how many gave identical answers? 
3. How many pairs had no answers in common?

Tip: it may help to first convert the NxN similarity matrix into a table with three columns: response_id_1, response_id_2, and the similarity between them. The new table should have NxN rows and 3 columns.

In [5]:
# convert the NxN similarity matrix into a
# table with three columns:
# response_id_1, response_id_2, and the similarity between them
tall_dist_df = pd.melt(dist_df.reset_index(), id_vars="index")
tall_dist_df.columns = ["student_1", "student_2", "hamming_dist"]

# removing redundant info
tall_dist_df = tall_dist_df[tall_dist_df["student_1"] < tall_dist_df["student_2"]]
print(tall_dist_df.shape)
tall_dist_df.head(3)

(4373403, 3)


,student_1,student_2,hamming_dist
2958,0,1,0.0
5916,0,2,0.1
5917,1,2,0.1


<IPython.core.display.Javascript object>

In [6]:
# How many pairs of students are there in our dataset?
# Thats a fancy way to say what is the value of "n choose k"
# in our case we have n=2958 students (q5.shape[0])
# and its asking about pairs (k=2)
# so we need to calculate "2958 choose 2"

# this is also the number of rows in how i created `tall_dist_df`
comb(2958, 2)

4373403.0

<IPython.core.display.Javascript object>

In [7]:
# Of these pairs, how many gave identical answers?
# identical answers means a dist of zero
identical_pairs = tall_dist_df[tall_dist_df["hamming_dist"] == 0]
identical_pairs.shape[0]

136732

<IPython.core.display.Javascript object>

In [8]:
# How many pairs had no answers in common?
# all different answers means a hamming dist of 1
opposite_pairs = tall_dist_df[tall_dist_df["hamming_dist"] == 1]
opposite_pairs.shape[0]

291

<IPython.core.display.Javascript object>

For each student, calculate their average similarity with all other students. Then print the following:

1. On average, which student (i.e. which response_id) had the most similar responses to the survey population as a whole? What was the average similarity for this student, and what were their answers?
2. Repeat part 1, but for the student with the least similar responses to the survey population as a whole.
3. What do we notice about these students? What broader inferences can we make about the student population as a whole?

In [9]:
q5.shape

(2958, 10)

<IPython.core.display.Javascript object>

In [10]:
# Calculating average distance per column
# most 'average' student will have smallest average dist
# most 'unique' student will have largest average dist

# extra context:
# * theres 10 columns,
# * so matching on everything but one is 1/10 or 0.1
# * so matching on only one is 9/10 or 0.9
avg_dist = dist_df.mean().sort_values()

<IPython.core.display.Javascript object>

In [11]:
# Data is sorted, so first and last are most 'average' and 'unique'
most_average = avg_dist.head(1).index
most_unique = avg_dist.tail(1).index

<IPython.core.display.Javascript object>

In [12]:
# This person is only stressed about academics
q5.loc[most_average, :].T

,0
Q5-Stressed about Adjustment issues,0
Q5-Stressed about Academic issues,1
Q5-Stressed about Financial issues,0
Q5-Stressed about Family issues,0
Q5-Stressed about Friendships,0
Q5-Stressed about Romantic relationships,0
Q5-Stressed about Health related issues,0
Q5-Stressed about Career related issues,0
"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",0
Q5-Stressed about Others,0


<IPython.core.display.Javascript object>

In [13]:
# This person is really stressed...
q5.loc[most_unique, :].T

,2899
Q5-Stressed about Adjustment issues,1
Q5-Stressed about Academic issues,1
Q5-Stressed about Financial issues,1
Q5-Stressed about Family issues,1
Q5-Stressed about Friendships,1
Q5-Stressed about Romantic relationships,0
Q5-Stressed about Health related issues,1
Q5-Stressed about Career related issues,1
"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",1
Q5-Stressed about Others,1


<IPython.core.display.Javascript object>

In [14]:
# The student most similar to everyone is only stressed
# about academics; this implies, that overall students are
# mostly stressed about academics and not much else.

# This is supported by the data below. Academics is the
# only item that >50% of the respondants said they were
# stressed about.  The most unique student is the most
# unique in that they are stressed about almost everything
# on the survey.

# A supporting calculation for interpretation:
pd.DataFrame(q5.mean().sort_values(), columns=["Average Response"])

,Average Response
Q5-Stressed about Others,0.013861
Q5-Stressed about Family issues,0.147397
Q5-Stressed about Health related issues,0.151116
Q5-Stressed about Romantic relationships,0.165652
Q5-Stressed about Friendships,0.269777
"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",0.274172
Q5-Stressed about Adjustment issues,0.306626
Q5-Stressed about Financial issues,0.386072
Q5-Stressed about Career related issues,0.455037
Q5-Stressed about Academic issues,0.922245


<IPython.core.display.Javascript object>